In [1]:
import numpy as np
import math
import h5py
import matplotlib.pyplot as plt
import pandas as pd
import pandas_datareader.data as web
from datetime import datetime, timedelta
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from keras.models import Sequential, load_model
from keras.layers import Dense, LSTM

Using TensorFlow backend.


In [2]:
# Importing the dataset
stock = 'GOOGLE'
symbol = 'GOOGL'
start_date = datetime.today()+timedelta(days=-2000)
end_date = datetime.today()
prices = web.DataReader(symbol, 'yahoo', start_date, end_date)
print prices.shape

# Averaging the Open, High, Low, Close Prices for each day
prices = prices.iloc[:,:4].values.mean(axis=1)

# Normalizing the average price
sc = MinMaxScaler()
prices = sc.fit_transform(prices)


(1380, 6)


/usr/local/lib/python2.7/dist-packages/sklearn/preprocessing/data.py:321: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)
/usr/local/lib/python2.7/dist-packages/sklearn/preprocessing/data.py:356: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)


In [3]:

def structure_time_series(prices, history_size, forecast_size):
    '''
    Converts the time series into a structured format for training purposes
    Days(t-history_size, t-1) will be the input variables
    Days(t, t+forecast_size) will be the target variables
    '''
    
    df = pd.DataFrame(prices)
    features, names = [], []

    for i in range(history_size, 0, -1):
        features.append(df.shift(i))
        names += [('day t-%d' % (i))]

    for i in range(0, forecast_size):
        features.append(df.shift(-i))
        names += [('day t+%d' % (i))]
        

    data = pd.concat(features, axis=1)
    data.columns = names
    
    # Slicing the dataframe to avoid null values
    return data[history_size:-forecast_size+1]


In [4]:

def train_test_split(prices, split, history_size, forecast_size):
    values = prices.values
    n_train_hours = int(prices.shape[0] * split)
    train = values[:n_train_hours, :]
    test = values[n_train_hours:, :]
    
    # split into input and outputs
    X_train, y_train = train[:, :history_size], train[:, -forecast_size-1:-1]
    X_test, y_test = test[:, :history_size], test[:, -forecast_size-1:-1]
    
    # reshape input to be 3D [samples, timesteps, features]
    X_train = X_train.reshape((X_train.shape[0], 1, X_train.shape[1]))
    X_test = X_test.reshape((X_test.shape[0], 1, X_test.shape[1]))
    y_train = y_train.reshape((y_train.shape[0], y_train.shape[1]))
    y_test = y_test.reshape((y_test.shape[0], y_test.shape[1]))

    return X_train, y_train, X_test, y_test

In [5]:
def build_and_train_model(X_train, y_train):
    model = Sequential()
    model.add(LSTM(64, input_shape=(X_train.shape[1], X_train.shape[2])))
    model.add(Dense(y_train.shape[1]))
    model.compile(loss='mse', optimizer='adam')

    model.fit(X_train, y_train, epochs=50, batch_size=32, validation_split=0.1)

    return model

In [ ]:
history_size = 30

##### 2-Day Forecast #####

data = structure_time_series(prices, history_size, 2)
X_train, y_train, X_test, y_test = train_test_split(data, 0.9, history_size, 2)
print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)


((1214, 1, 30), (1214, 2), (135, 1, 30), (135, 2))


In [ ]:
model1 = build_and_train_model(X_train, y_train)

pred = model1.predict(X_test)
rmse = math.sqrt(mean_squared_error(y_test, pred))
print('Test RMSE: %.3f' % rmse)

model1.save('2DayForecast.h5')

In [ ]:
##### 5-Day Forecast #####

data = structure_time_series(prices, history_size, 5)
X_train, y_train, X_test, y_test = train_test_split(data, 0.9, history_size, 5)
print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)



In [ ]:
model2 = build_and_train_model(X_train, y_train)

pred = model2.predict(X_test)
rmse = math.sqrt(mean_squared_error(y_test, pred))
print('Test RMSE: %.3f' % rmse)

model2.save('5DayForecast.h5')

In [ ]:
##### 10-Day Forecast #####

data = structure_time_series(prices, history_size, 10)
X_train, y_train, X_test, y_test = train_test_split(data, 0.9, history_size, 10)
print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)



In [ ]:
model3 = build_and_train_model(X_train, y_train)

pred = model3.predict(X_test)
rmse = math.sqrt(mean_squared_error(y_test, pred))
print('Test RMSE: %.3f' % rmse)

model3.save('10DayForecast.h5')